# Library Testing! 

Start by testing interpolation against existing UR3 library

1. Import + initialize Liam and Aengus libray 

In [ ]:
from UR_summer2025 import MyUR3e, Trajectory 

# Create an instance of the MyUR3e class
robot = MyUR3e()
traj = Trajectory()
print("done")

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!ros2 topic list

2. Import Hannah and Leia library + create a test path

In [ ]:
import time
import numpy as np 

# Test path
traj.trajectory = [[0,0,0,0,0,0],
                    [.03,.04,.01,30,0,0],
                    [.06,.08,.02,90,20,0],
                    [.06,.08,.02,90,20,20]]
#testing no args
traj.linear_interp() # should default interpolate traj.trajectory w/ step size .01
print(traj.trajectory)

print("done")

You can check the nodes by running:

In [ ]:
!ros2 topic list

This should be a working path that throws no errors and prints results. 

In [ ]:
traj.trajectory = [[0,0,0,0,0,0],
                    [3,4,1,30,0,0],
                    [6,8,2,90,20,0],
                    [6,8,2,90,20,20]]
interpolated_result = traj.linear_interp(step_size = 1, debugging=True, aligned=False)

This path should throw no errors but no interpolation should occur (Original = Interpolated)

In [ ]:
# nothing should happen here
zeros_path = [[0,0,0,0,0,0],
            [0,0,0,0,0,40],
            [0,0,0,0,0,0]]
traj.linear_interp(zeros_path, step_size=1, debugging=True, aligned=True)

All of the following should throw errors for invalid input. 

In [ ]:
# Error Testing
no_path = []
path_too_short = [[0,0,0,0,0,0]]

traj.linear_interp(no_path, step_size=1, debugging=True)
traj.linear_interp(zeros_path, step_size=0, debugging=True)
traj.linear_interp(path_too_short, step_size=1, debugging=True)

## Relative Trajectory testing: 

In [ ]:
# Testing relative trajectory
relative_trajectory = np.array([
    [0.0, 0.0, -0.01, 0.0, 0.0, 0.0],
    [0.1, 0.0, 0.0, 0.0, 0.0, 90.0],
])
# relative_trajectory = linear_interp(relative_trajectory, debugging=True)
# Move the end effector through the trajectory
start = [0.2, 0.1, 0.4, 0.0, 0.0, 0.0]
global_trajectory = traj.relative_to_global(relative_trajectory, start)

print(f"global: {global_trajectory}")

## Testing with the arm: 

In [ ]:
# Move in a 10cm square using global relative commands
relative_trajectory = [
    [0.0, 0.0, -0.01, 0.0, 0.0, 0.0],
    [0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 90.0],
    [0.0, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, -90.0],
    [-0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0, 0.0, 0.0, 0.0, 0.0, 90.0],
    [0.0, -0.1, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, -90.0]
]
start_pos = [0.2, 0.1, 0.4, 0.0, 0.0, 0.0]
global_trajectory = traj.relative_to_global(relative_trajectory, start_pos)
global_trajectory = traj.linear_interp(global_trajectory, debugging=False, aligned=False)
# Move the end effector through the trajectory
robot.move_global(start_pos)
robot.move_global(global_trajectory, (len(global_trajectory)/2))
print("done")

## Drawing Circles

In [ ]:
# center = [0.4, 0.2, 0.5, 0, 0, 0] # generated location
center = [0.25, 0.1, 0.34, 0.0, 0.0, 0.0]
robot.move_global(center)
print("make sure center position seems viable.")
trajectory = traj.draw_circle(
    center,
    radius=0.1,
    num_points=24,
    degrees=True,
    plane_rotation=[0, 0, 0],  # Tilt circle 45° around Y-axis
    debugging=False,
    aligned=True
)
robot.move_global(trajectory, len(trajectory)/2)
print("done")

## Zig Zag

In [ ]:
robot.read_global_pos()

In [ ]:
zig_zag = [[.26, -0.02, 0, 0, 0, 0],
           [.2, -0.01, 0, 0, 0, 0],
           [.26, 0, 0, 0, 0, 0],
           [.2, 0.01, 0, 0, 0, 0],
           [.26, 0.02, 0, 0, 0, 0],
           [.2, 0.03, 0, 0, 0, 0],
           [.26, 0.04, 0, 0, 0, 0],
           [.2, 0.03, 0, 0, 0, 0],
           [.26, 0.02, 0, 0, 0, 0],
           [.2, 0.01, 0, 0, 0, 0],
           [.26, 0, 0, 0, 0, 0],
           [.2, -0.01, 0, 0, 0, 0],
           [.26, -0.02, 0, 0, 0, 0]]
traj.trajectory = zig_zag
traj.set_z(.25)
start = [.28, -.02, .3, 0, 0, 0]
robot.move_global(start)
time.sleep(2)
traj.linear_interp()
robot.move_global(traj.trajectory, 15, wait=True)
robot.move_global([.26, -0.02, .2, 0, 0, 0])
print("done")

## Clay Command testing

In [ ]:
from UR_summer2025 import ClaySculpt, ClayTrajectory
import time
clay = ClaySculpt()
clay_traj = ClayTrajectory()
print("done")

#### testing inheritance:

In [ ]:
# clay.move_global([0.33, 0.05, 0.19, 0.0, 0, 0.0])
# clay.move_gripper(95)
# clay.move_gripper(83)
clay.move_gripper(25, force=75)
# 28 for wood tool, 75 for sheetmetal

In [ ]:
clay.move_global([0.4, 0.1, 0.3, 30.0, 0, -90.0])
# start = [.3, -.03, .35, 0, 0, 0]
# clay.move_global(start)
# clay.move_global([0.33, 0.05, 0.3, 0.0, 0, 0.0])

In [ ]:
# testing roll sequence
clay_traj.roll_sequence([.225,0])  
clay_traj.set_z(.20)
start = [.3, -.03, .2, 0, 0, 0]
clay.move_global(start)
time.sleep(2)
clay.move_global(clay_traj.trajectory, 15, wait=True)
clay.move_global([.26, -0.02, .2, 0, 0, 0])
print("done")

In [ ]:
# testing relative trajectories 
print(clay_traj.trajectory)
clay_traj.relative_to_global([[0, 0, -.02, 0, 0, 0]], [0.25, 0.09999, 0.3000, 0.1006, 0.1011, 0.09808])
print(clay_traj.trajectory)

## MUST include the double brackets

In [ ]:
# testing blend trajectory. 
# x coordinate cannot exceed .45 (no IK solution)
clay.move_global([0.4, 0.1, 0.25, 30.0, 0, -90.0])
clay_traj.blend_traj([.2, -.16], cycles=2, angle = -30, width=.005, diameter=.228)
clay.move_global(clay_traj.trajectory, 5)
clay_traj.blend_traj([.2, .07], cycles=2, angle = 30, width=.005, diameter=.228)
clay.move_global(clay_traj.trajectory, 5)

In [ ]:
# sponge
import numpy as np
clay_traj.blend_traj([.2, 0], cycles=4, angle = 0, length = .2, width=.01, diameter=.195)
offset = [.22, 0, 0, 0, 0, 0]
print(clay_traj.trajectory[2])
print(len(clay_traj.trajectory))
R = clay_traj.euler_rotation_matrix(0, 0, 90)
for i in range(len(clay_traj.trajectory)):
    for j in range(6):
        clay_traj.trajectory[i][j] = clay_traj.trajectory[i][j] - offset[j]
    pos = clay_traj.trajectory[i][:3]
    new_pos = R @ pos
    new_pos[0] = new_pos[0] + .22
    clay_traj.trajectory[i][:3] = new_pos
print(clay_traj.trajectory[2])
# clay.move_global(clay_traj.trajectory, 20)

In [ ]:
# testing blend trajectory
# max .45 x 
clay.move_global([0.4, 0.1, 0.235, 30.0, 0, -90.0])
clay.blend(height=.228, length=.2, offset_1=-.16, offset_2=.07, pass_width=.0025)

In [ ]:
# testing Z calibration
print(clay.zero_force)
clay.calibrate_z()
print(clay.zero_force)

In [ ]:
# testing roll coil (w/ force detection)
clay.move_global([0.33, 0.05, 0.35, 0.0, 0, 0.0])
# time.sleep(5) 
clay.move_global([0.33, 0.05, 0.22, 0.0, 0, 0.0])
clay.roll_coil(.20, step=.0015, sweep=False, force_based=True, force_threshold = 15, z_test = False)
print(clay.force)
print(clay.height)

In [ ]:
clay.read_global_pos()